In [2]:
pip install pyspark

  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824030 sha256=79a5ffc7c4f5dac344829be1582e40961d9296bf4de1f6412d23ec2b9d4d2fb4
  Stored in directory: c:\users\pc\appdata\local\pip\cache\wheels\b1\59\a0\a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark
  Using cached py4j-0.10.9.5-py2.py3-none-any.whl (199 kB)
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824030 sha256=4eb150e0b55df483a95d6098dbb7fafcd5bc0076385403955d7743b794df83bf
  Stored in directory: c:\users\pc\appdata\local\pip\cache\wheels\b1\59\a0\a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark



In [8]:
#This is needed to start a Spark session from the notebook
#You may adjust the memory used by the driver program based on your machine's settings
import os 
os.environ['PYSPARK_SUBMIT_ARGS'] ="--conf spark.driver.memory=2g  pyspark-shell"

from pyspark.sql import SparkSession

print(1)

1


In [9]:
# -------------------------------
# Start Spark in LOCAL mode
# -------------------------------

#The following lines are just there to allow this cell to be re-executed multiple times:
#if a spark session was already started, we stop it before starting a new one
#(there can be only one spark context per jupyter notebook)
try: 
    spark
    print("Spark application already started. Terminating existing application and starting new one")
    spark.stop()
except: 
    pass

# Create a new spark session (note, the * indicates to use all available CPU cores)
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("demoRDD") \
    .getOrCreate()
    
#When dealing with RDDs, we work the sparkContext object. See https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext
sc=spark.sparkContext

Spark application already started. Terminating existing application and starting new one


In [11]:
#from google.colab import drive
#drive.mount('/content/drive')
print(1)

1


In [12]:
# Create an DataFrame from drivers.csv
drivers = spark.read.csv('drivers.csv', header=True, inferSchema=True)

In [13]:
df1 = spark.read.json('zones.json')


In [14]:
#Exercise 1
drivers.show()

+--------+-------------------+----------+----------+
|  driver|          timestamp|  latitude| longitude|
+--------+-------------------+----------+----------+
|c473205b|2017-08-31 14:24:25|-12.106778|-76.998078|
|a0f3b4e1|2017-08-31 14:24:26|-12.103913|-76.963727|
|1236f9fe|2017-08-31 14:24:26|-12.133777|-77.004266|
|ae4a06a2|2017-08-31 14:24:26|-12.085963|-76.987582|
|ab7a6c63|2017-08-31 14:24:26|-12.072973|-77.061448|
|5ee73484|2017-08-31 14:24:26|-12.067694|-77.068442|
|4fff90cb|2017-08-31 14:24:26|-12.144308|-76.989234|
|d892d208|2017-08-31 14:24:26|-16.401221|-71.499513|
|e9f90dfb|2017-08-31 14:24:10|-12.063665|-76.963254|
|c1719f8d|2017-08-31 14:24:10|-12.070187|-76.994167|
|4c299505|2017-08-31 14:24:11| -11.96459|-77.015983|
|49f033bd|2017-08-31 14:24:11|  -12.0906|-77.069808|
|749df32a|2017-08-31 14:24:11|-12.109964| -76.97535|
|f8d5c453|2017-08-31 14:24:11|-12.080683|-77.036188|
|4314c58a|2017-08-31 14:24:11|-16.404571|-71.519775|
|914f7296|2017-08-31 14:24:11|-12.218541|-76.9

In [15]:
df1.show()

+--------------------+
|               zones|
+--------------------+
|[{1, [{-11.784234...|
+--------------------+



In [16]:
# Create an DataFrame from zones.json
from pyspark.sql.functions import explode
zones = spark.read.json('zones.json')
zones=zones.withColumn('zones',explode('zones'))
zones=zones.select('zones.id_zone',explode('zones.polygon').alias('Coordinates'))
zones=zones.select('id_zone','Coordinates.lat', 'Coordinates.lng')

In [17]:
zones.show()

+-------+-------------------+------------------+
|id_zone|                lat|               lng|
+-------+-------------------+------------------+
|      1|-11.784234676000215|-77.28648789023627|
|      1|-11.873627286463377|-77.28648789023627|
|      1|-11.869434808611365|-77.19452474207226|
|      1| -11.78007195481368|-77.19452474207226|
|      1|-11.784234676000215|-77.28648789023627|
|      2|-12.479157220410556|-76.82596521434296|
|      2|-12.569000505748466|-76.82596521434296|
|      2|-12.564395154012836| -76.7336521407365|
|      2|-12.474582671650962| -76.7336521407365|
|      2|-12.479157220410556|-76.82596521434296|
|      3|-12.201069396462607|-76.64127121444726|
|      3| -12.29062991993957|-76.64127121444726|
|      3| -12.28604810560784|  -76.548823150976|
|      3|  -12.1965189187836|  -76.548823150976|
|      3|-12.201069396462607|-76.64127121444726|
|      4|-12.205585634694005| -76.7336521407365|
|      4|-12.295177256777961| -76.7336521407365|
|      4| -12.290629

In [18]:
from pyspark.sql import functions as f

In [19]:
# We assume that a driver is inside a zone when its coordinates are included in the intervals formed by the minimum and maximum of the latitude and longitude of the polygon inside which it is located.
zonesdf = zones.groupby("id_zone").agg(f.max("lat").alias("maxlat"),
                                        f.min("lat").alias("minlat"),
                                        f.max("lng").alias("maxlng"),
                                        f.min("lng").alias("minlng"))

In [20]:
zonesdf.show()

+-------+-------------------+-------------------+------------------+------------------+
|id_zone|             maxlat|             minlat|            maxlng|            minlng|
+-------+-------------------+-------------------+------------------+------------------+
|     26|-11.860950432726074|-11.954619512552217|-77.01038495147722|-77.10249019404858|
|     29|-11.771647803699995| -11.86520920024759|-77.01038495147722|-77.10249019404858|
|     19|-11.941652263135907|-12.035434603017723|-76.82596521434296|-76.91820972175559|
|     22|-12.039787447923926|-12.133669908128265|-77.01038495147722|-77.10249019404858|
|      7|-12.205585634694005|-12.299690140508726| -76.7336521407365|-76.82596521434296|
|     34|-11.758764395010498| -11.85233331092545| -76.7336521407365|-76.82596521434296|
|     50|-11.690787182377083|-11.784234676000215|-77.19452474207226|-77.28648789023627|
|     32| -11.85233331092545|-11.946008121670031|-76.82596521434296|-76.91820972175559|
|     43| -11.58907701296681|-11

In [21]:
# Join the DataFrames
# Thus we were able to assign the drivers to the zones according to their coordinates and find the 10 most frequented zones.
from pyspark.sql.functions import array_contains, round,max,min
drivers_with_zones = drivers.join(zonesdf,(drivers.latitude.between(zonesdf.minlat,zonesdf.maxlat)) & (drivers.longitude.between(zonesdf.minlng,zonesdf.maxlng)), how='outer')


In [22]:
drivers_with_zones.filter(drivers_with_zones.id_zone.isNotNull()).show()

+--------+-------------------+----------+----------+-------+-------------------+-------------------+------------------+------------------+
|  driver|          timestamp|  latitude| longitude|id_zone|             maxlat|             minlat|            maxlng|            minlng|
+--------+-------------------+----------+----------+-------+-------------------+-------------------+------------------+------------------+
|ab7a6c63|2017-08-31 14:24:26|-12.072973|-77.061448|     22|-12.039787447923926|-12.133669908128265|-77.01038495147722|-77.10249019404858|
|5ee73484|2017-08-31 14:24:26|-12.067694|-77.068442|     22|-12.039787447923926|-12.133669908128265|-77.01038495147722|-77.10249019404858|
|4c299505|2017-08-31 14:24:11| -11.96459|-77.015983|     25| -11.95033052922269|-12.044106631207434|-77.01038495147722|-77.10249019404858|
|49f033bd|2017-08-31 14:24:11|  -12.0906|-77.069808|     22|-12.039787447923926|-12.133669908128265|-77.01038495147722|-77.10249019404858|
|f8d5c453|2017-08-31 14:24:

In [23]:
drivers_with_zones.groupby('id_zone').count().orderBy('count', ascending=False).filter(drivers_with_zones.id_zone.isNotNull()).show(10)

+-------+-------+
|id_zone|  count|
+-------+-------+
|     22|1997917|
|     21|1029105|
|     14| 338673|
|     25| 252043|
|     13| 156504|
|     20|  81069|
|     23|  55152|
|     24|  49578|
|     26|  48650|
|     15|  18404|
+-------+-------+
only showing top 10 rows



Discuss how Spark parallelizes the computation

Parallelization is a method of creating an RDD from an existing collection or array in a submersible. Items in the collection are copied to form distributed datasets that we can exploit in parallel.

This means that spark divides data frames into smaller datasets, distributes and converts them into pandas objects, and applies functions to them. Finally, the results are combined back into a large Spark dataframe.

In other words, by using the parallelization approach, Spark runs two or more parts of a single large data set as a separate process, reducing processing time and improving performance.

Parallelization algorithm:

1. Spark runs the program and submits the job using "Spark-submit", which is used to launch the program on the cluster.
2. After the job is submitted, the job is forwarded to the SparkContext driver. The Spark context driver is the ingress port for Spark.
3. SC routes the program to modules such as the cluster master node and RDD created by the Sparkcontext driver.
4. The program is then provided to the cluster master node (including the executor), each cluster has a master node, performing all the necessary processing. It forwards the program further to the worker node (to solve the problem).

In [24]:
#Exercise 2
drivers.select(['driver', 'timestamp', 'latitude', 'longitude']).show()

+--------+-------------------+----------+----------+
|  driver|          timestamp|  latitude| longitude|
+--------+-------------------+----------+----------+
|c473205b|2017-08-31 14:24:25|-12.106778|-76.998078|
|a0f3b4e1|2017-08-31 14:24:26|-12.103913|-76.963727|
|1236f9fe|2017-08-31 14:24:26|-12.133777|-77.004266|
|ae4a06a2|2017-08-31 14:24:26|-12.085963|-76.987582|
|ab7a6c63|2017-08-31 14:24:26|-12.072973|-77.061448|
|5ee73484|2017-08-31 14:24:26|-12.067694|-77.068442|
|4fff90cb|2017-08-31 14:24:26|-12.144308|-76.989234|
|d892d208|2017-08-31 14:24:26|-16.401221|-71.499513|
|e9f90dfb|2017-08-31 14:24:10|-12.063665|-76.963254|
|c1719f8d|2017-08-31 14:24:10|-12.070187|-76.994167|
|4c299505|2017-08-31 14:24:11| -11.96459|-77.015983|
|49f033bd|2017-08-31 14:24:11|  -12.0906|-77.069808|
|749df32a|2017-08-31 14:24:11|-12.109964| -76.97535|
|f8d5c453|2017-08-31 14:24:11|-12.080683|-77.036188|
|4314c58a|2017-08-31 14:24:11|-16.404571|-71.519775|
|914f7296|2017-08-31 14:24:11|-12.218541|-76.9

In [25]:
drivers= drivers.select(f.col("driver"),f.col("latitude"),f.col("longitude"),f.to_timestamp(f.col("timestamp")).alias("timestamp"))
drivers.show()


+--------+----------+----------+-------------------+
|  driver|  latitude| longitude|          timestamp|
+--------+----------+----------+-------------------+
|c473205b|-12.106778|-76.998078|2017-08-31 14:24:25|
|a0f3b4e1|-12.103913|-76.963727|2017-08-31 14:24:26|
|1236f9fe|-12.133777|-77.004266|2017-08-31 14:24:26|
|ae4a06a2|-12.085963|-76.987582|2017-08-31 14:24:26|
|ab7a6c63|-12.072973|-77.061448|2017-08-31 14:24:26|
|5ee73484|-12.067694|-77.068442|2017-08-31 14:24:26|
|4fff90cb|-12.144308|-76.989234|2017-08-31 14:24:26|
|d892d208|-16.401221|-71.499513|2017-08-31 14:24:26|
|e9f90dfb|-12.063665|-76.963254|2017-08-31 14:24:10|
|c1719f8d|-12.070187|-76.994167|2017-08-31 14:24:10|
|4c299505| -11.96459|-77.015983|2017-08-31 14:24:11|
|49f033bd|  -12.0906|-77.069808|2017-08-31 14:24:11|
|749df32a|-12.109964| -76.97535|2017-08-31 14:24:11|
|f8d5c453|-12.080683|-77.036188|2017-08-31 14:24:11|
|4314c58a|-16.404571|-71.519775|2017-08-31 14:24:11|
|914f7296|-12.218541|-76.928484|2017-08-31 14:

In [27]:
drivers_with_zones = drivers_with_zones.select(f.col("driver"),f.col("latitude"),f.col("longitude"),f.col("timestamp"),f.col('id_zone'),f.col('maxlat'),f.col('minlat'),f.col('maxlng'),f.col('minlng'))
print(drivers_with_zones.dtypes)


[('driver', 'string'), ('latitude', 'double'), ('longitude', 'double'), ('timestamp', 'timestamp'), ('id_zone', 'bigint'), ('maxlat', 'double'), ('minlat', 'double'), ('maxlng', 'double'), ('minlng', 'double')]


In [28]:
drivers_with_zones.show()

+--------+----------+----------+-------------------+-------+-------------------+-------------------+------------------+------------------+
|  driver|  latitude| longitude|          timestamp|id_zone|             maxlat|             minlat|            maxlng|            minlng|
+--------+----------+----------+-------------------+-------+-------------------+-------------------+------------------+------------------+
|c473205b|-12.106778|-76.998078|2017-08-31 14:24:25|     21|-12.035434603017723|-12.129320541365702|-76.91820972175559|-77.01038495147722|
|a0f3b4e1|-12.103913|-76.963727|2017-08-31 14:24:26|     21|-12.035434603017723|-12.129320541365702|-76.91820972175559|-77.01038495147722|
|1236f9fe|-12.133777|-77.004266|2017-08-31 14:24:26|     14|-12.124937275664365|-12.218929159905016|-76.91820972175559|-77.01038495147722|
|ae4a06a2|-12.085963|-76.987582|2017-08-31 14:24:26|     21|-12.035434603017723|-12.129320541365702|-76.91820972175559|-77.01038495147722|
|ab7a6c63|-12.072973|-77.06

In [29]:
drivers_with_zones.withColumn("test", f.col("latitude")*2).show()


+--------+----------+----------+-------------------+-------+-------------------+-------------------+------------------+------------------+----------+
|  driver|  latitude| longitude|          timestamp|id_zone|             maxlat|             minlat|            maxlng|            minlng|      test|
+--------+----------+----------+-------------------+-------+-------------------+-------------------+------------------+------------------+----------+
|c473205b|-12.106778|-76.998078|2017-08-31 14:24:25|     21|-12.035434603017723|-12.129320541365702|-76.91820972175559|-77.01038495147722|-24.213556|
|a0f3b4e1|-12.103913|-76.963727|2017-08-31 14:24:26|     21|-12.035434603017723|-12.129320541365702|-76.91820972175559|-77.01038495147722|-24.207826|
|1236f9fe|-12.133777|-77.004266|2017-08-31 14:24:26|     14|-12.124937275664365|-12.218929159905016|-76.91820972175559|-77.01038495147722|-24.267554|
|ae4a06a2|-12.085963|-76.987582|2017-08-31 14:24:26|     21|-12.035434603017723|-12.129320541365702|

In [31]:
drivers_with_zones.withColumn("year", f.year(f.col("timestamp")).show()

SyntaxError: unexpected EOF while parsing (<ipython-input-31-5cb9d7630e75>, line 1)

In [ ]:
drivers_with_zones = drivers_with_zones.withColumn("year", f.year(f.col("to_timestamp(timestamp)")))
drivers_with_zones = drivers_with_zones.withColumn("month", f.month(f.col("to_timestamp(timestamp)")))
drivers_with_zones = drivers_with_zones.withColumn("day", f.dayofweek(f.col("to_timestamp(timestamp)")))
drivers_with_zones.show()

In [ ]:
drivers_with_zones.groupBy(["id_zone","month"]).count().show()

In [ ]:
test = drivers_with_zones.groupBy(["id_zone","month"]).count()
print((test.count(), len(test.columns)))


In [ ]:
import pandas as pd    
#spark.conf.set("spark.sql.execution.arrow.enabled", "true")
month_pandasDF=test.toPandas()
print(month_pandasDF.shape)

In [ ]:
month_pandasDF.head()


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def get_plot(sparkdf,groupby_col):
  sparkdf = sparkdf.groupBy(groupby_col).count()
  # Convert to pandas.DataFrame
  pandas_df=sparkdf.toPandas()
  print("Dataframe shape : ",pandas_df.shape)
  # count normalization
  pandas_df["normalize_count"] = pandas_df["count"]/sum(pandas_df["count"])
  # plot
  plt.figure(figsize=(12,8))
  plt.bar(x = pandas_df["id_zone"], height = pandas_df["normalize_count"])
  plt.xlabel("id_zone")
  plt.ylabel(groupby_col[1])
  plt.title(f"Distribution by Zones for {groupby_col[1]}")

  plt.show()

In [ ]:
get_plot(drivers_with_zones,["id_zone","month"])

In [ ]:
get_plot(drivers_with_zones,["id_zone","year"])


In [ ]:
def get_data(sparkdf,groupby_col):
  sparkdf = sparkdf.groupBy(groupby_col).count()
  # Convert to pandas.DataFrame
  pandas_df=sparkdf.toPandas()
  print("Dataframe shape : ",pandas_df.shape)
  # count normalization
  pandas_df["normalize_count"] = pandas_df["count"]/sum(pandas_df["count"])
  return pandas_df

In [ ]:
get_data(drivers_with_zones,["id_zone","day"])

Comment:The distribution shows that there are three categories of zones:
high traffic zones (22 and 23);
medium affluence zones (13, 14 and 26) and the rest are very low affluence zones.
What additional knowledge would you like to extract from this data

In this data we may exact the time and date.

 As before, comment on the parallelization strategy.

Parallelize is a method to create an RDD from an existing collection or array present in the driver. The elements present in the collection are copied to form a distributed dataset on which we can operate on in parallel.